In [12]:
import numpy as np
import tensorflow as tf
import keras

ImportError: cannot import name 'Tensor' from 'tensorflow.python.framework.ops' (/home/vscode/.local/lib/python3.10/site-packages/tensorflow/python/framework/ops.py)

In [ ]:
import os
import random
import shutil
import os
import matplotlib.pyplot as plt
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [5]:
# Define paths
data_path = './data/dogs-vs-cats/train/'  # Update the path if needed
output_path = './data/sample_images/'
os.makedirs(output_path, exist_ok=True)

In [6]:
# List all image files in the directory
all_images = os.listdir(data_path)

FileNotFoundError: [Errno 2] No such file or directory: './data/dogs-vs-cats/train/'

In [ ]:
# Separate cat and dog images
cat_images = [img for img in all_images if img.startswith('cat')]
dog_images = [img for img in all_images if img.startswith('dog')]

In [ ]:
# Randomly select 100 cat and 100 dog images
selected_cats = random.sample(cat_images, 100)
selected_dogs = random.sample(dog_images, 100)


In [ ]:
# Copy selected images to the output folder
for img in selected_cats + selected_dogs:
    shutil.copy(os.path.join(data_path, img), os.path.join(output_path, img))

print(f"Copied 200 random cat and dog images to {output_path}")


In [ ]:
# Paths to the image directories
data_path = './data/dogs-vs-cats/train'

In [ ]:
# Get the first 9 dog and cat images
dog_images = [img for img in os.listdir(data_path) if 'dog' in img][:9]
cat_images = [img for img in os.listdir(data_path) if 'cat' in img][:9]

In [ ]:
# Function to plot images in a grid
def plot_images(images, title):
    plt.figure(figsize=(10, 10))
    for i, img in enumerate(images):
        plt.subplot(3, 3, i + 1)
        img = cv2.imread(os.path.join(data_path, img))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (200, 200))
        plt.imshow(img)
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

In [ ]:
# Plot first 9 dogs and cats
plot_images(dog_images, 'Dogs')
plot_images(cat_images, 'Cats')

In [ ]:
# Data augmentation and resizing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # 20% for validation

train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(200, 200),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
model = Sequential()

# Convolution layers
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu', input_shape=(200, 200, 3)))
model.add(Conv2D(64, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

# Flattening and Dense layers
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Binary classification

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, mode='min')

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[checkpoint, early_stopping]
)

In [ ]:
# Load the best saved model
from keras.models import load_model

best_model = load_model('best_model.h5')

# Evaluate on validation data
val_loss, val_acc = best_model.evaluate(validation_generator)
print(f'Validation Accuracy: {val_acc * 100:.2f}%')

In [ ]:
# Save the model architecture and weights
best_model.save('dogs_vs_cats_model.h5')